In [64]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import time
import numpy as np
import lxml
import chardet

In [66]:
ligaMxUrl = 'https://fbref.com/en/comps/31/Liga-MX-Stats'


In [67]:
def getLeagueTeamsUrls(ligaMxUrl):
    data = req.get(ligaMxUrl)
    #soup = BeautifulSoup(data.text, features = "html.parser")
    soup = BeautifulSoup(data.text, "lxml")

    standings_table=soup.select('table.stats_table')[0]

    #Retrieving team links
    links = standings_table.find_all('a')
    links =  [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]

    league_urls = [f"https://fbref.com{l}" for l in links]
    return league_urls, soup


In [68]:
def getMatches(team_url):
    team_web_data = req.get(team_url)
    team_matches = pd.read_html(team_web_data.text, match = "Scores & Fixtures")[0]
    return team_web_data, team_matches

In [69]:
def getShooting(team_data):
    #soup = BeautifulSoup(team_data.text, features = "html.parser")
    soup = BeautifulSoup(team_data.text, 'lxml')

    shooting_link = soup.find_all("a")
    shooting_link = [l.get("href") for l in shooting_link]
    shooting_link = [l for l in shooting_link if l and 'all_comps/shooting/' in l]

    shooting_data = req.get(f"https://fbref.com{shooting_link[0]}")
    shootingInfo = pd.read_html(shooting_data.text, match='Shooting')[0]

    #transformating shooting info
    shootingInfo.columns = shootingInfo.columns.droplevel()
    return shootingInfo



In [70]:
#league_urls=['https://fbref.com/en/squads/c02b0f7a/Guadalajara-Stats']

years = list(range(2023, 2017, -1))
#years = [2023]

# List of dataframes: Dataframes with each team's matches information per season
all_matches = []

for year in years: 
    league_urls, seasonSoup = getLeagueTeamsUrls(ligaMxUrl)
    
    # Find anchor element with class 'prev' which contains link to previous season
    previous_season = seasonSoup.select("a.prev")[0].get("href")
    ligaMxUrl = f"https://fbref.com{previous_season}"

    for team_url in league_urls:
        teamWebData, teamMatchesDf = getMatches(team_url)
        teamShootingDf = getShooting(teamWebData)
        #time.sleep(1)
        
        try:
            
            team_df = teamMatchesDf.merge(teamShootingDf[["Date","Sh","SoT", "Dist","FK","PK","PKatt"]], on = "Date")
        except ValueError:
            continue

        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        team_df["team_name"] = team_name
        team_df["season"] = year
        all_matches.append(team_df)
        time.sleep(1)   

#team_df.head()


KeyError: "['FK'] not in index"

In [46]:
league_matches = pd.concat(all_matches)

In [47]:
league_matches.columns = [c.lower() for c in league_matches.columns] 


In [48]:
#league_matches["gf"] = league_matches["gf"].astype(float)
league_matches["gf"] = pd.to_numeric(league_matches["gf"], errors = 'coerce')
league_matches["ga"] = pd.to_numeric(league_matches["ga"], errors = 'coerce')


In [49]:
league_matches["gf"] =league_matches["gf"].astype('Int64')
league_matches["ga"] =league_matches["ga"].astype('Int64')

league_matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,team_name,season
0,2022-07-03,19:05,Liga MX,Apertura 2022 Regular Season,Sun,Away,L,3,4,Santos,...,Match Report,NaN,16,7,19.0,1,1,1,Monterrey,2023
1,2022-07-09,21:05,Liga MX,Apertura 2022 Regular Season,Sat,Home,W,3,2,América,...,Match Report,NaN,19,8,20.7,0,0,0,Monterrey,2023
2,2022-07-17,17:00,Liga MX,Apertura 2022 Regular Season,Sun,Away,W,1,0,Atlético,...,Match Report,NaN,18,4,24.1,0,1,1,Monterrey,2023
3,2022-07-21,21:00,Liga MX,Apertura 2022 Regular Season,Thu,Away,W,3,0,Querétaro,...,Match Report,NaN,15,6,16.8,1,0,0,Monterrey,2023
4,2022-07-26,19:00,Liga MX,Apertura 2022 Regular Season,Tue,Home,W,1,0,Puebla,...,Match Report,NaN,22,7,17.7,0,0,0,Monterrey,2023


In [50]:
league_matches.shape

(500, 27)